<a name = "content"> </a>
# Лабораторная работа №1

## Оглавление:

----

### [1. Задание и вариант](#step1)

### [2. Подготовительная часть](#step2)

### [3. Аналитическое решение](#step3)

### [4. Явная конечно-разностная схема](#step4)

* [Теория](#step4.1)

* [Программная реализация](#step4.2)

* [Исследовательская часть](#step4.3)

### [5. Неявная конечно-разностная схема](#step5)

* [Теория](#step5.1)

* [Программная реализация](#step5.2)

* [Исследовательская часть](#step5.3)

### [6. Схема Кранка-Николсона](#step6)

* [Теория](#step6.1)

* [Программная реализация](#step6.2)

* [Исследовательская часть](#step6.3)


<a name = "step1"></a>
# Задание
[К оглавлению](#content)

Используя явную и неявную конечно-разностные схемы, а также схему Кранка - Николсона, решить начально-краевую задачу для дифференциального уравнения параболического типа. Осуществить реализацию трех вариантов аппроксимации граничных условий, содержащих производные: двухточечная аппроксимация с первым порядком, трехточечная аппроксимация со вторым порядком, двухточечная аппроксимация со вторым порядком. В различные моменты времени вычислить погрешность численного решения путем сравнения результатов с приведенным в задании аналитическим решением $U(x,t)$. Исследовать зависимость погрешности от сеточных параметров $\tau, h$ .

## Вариант 7
Начально-краевая задача для дифференциального уравнения параболического типа.
$$\begin{align*} &\frac{\partial u}{\partial t} = \frac{\partial^2 u}{\partial x^2} + 0.5\exp(-0.5t)\cos x,\\
&u_x(0,t) = \exp(-0.5t),\\
&u_x(\pi,t) = -\exp(-0.5t),\\
&u(x,0) = \sin x,\end{align*}$$

Аналитическое решение $U(x,t) = \exp(-0.5t)\sin x.$

<a name = "step2"></a>
# Подготовительная часть

[К оглавлению](#content)

In [1]:
import math
import numpy as np
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm
co = 0

In [2]:
def analytic(x,t):
    return math.exp(-0.5*t)*math.sin(x)
def f_1(x,t):
    return 0.5*math.exp(-0.5*t)*math.sin(x)
def f_2(x,t):
    return math.exp(-0.5*t)
def f_3(x,t):
    return -f_2(x,t)
def f_4(x,t = None):
    return math.sin(x)
        
def MAE(x,t,u):
    error = 0
    for x_ in range(len(x)):
        for y_ in range(len(y)):
            error += abs(u[x_][y_] - analytic(x[x_], y[y_]))
    return error/(len(x) * len(y))    

def normm(x,t,u):
    errors = []
    for i in range(len(u)):
        maximum = 0
        for j in range(len(x)):
            tmp = abs(u[i][j] - analytic(x[j], t[i]))
            if maximum < tmp:
                maximum = tmp
        errors.append(maximum)
    return errors
def norm_error(x,t,u):
    ans = 0.0
    for i in range(len(u)):
        for j in range(len(u[i])):
            ans += (u[i][j] - analytic(x[i][j], t[i][j]))**2
    return ans/len(u[0])
    
def MSE(X,T,U):
    error = 0
    for x in range(0,len(X)):
        for t in range(0,len(T)):
            error += (U[t][x] - analytic(X[x],T[t]))**2
    return (error/(len(X)*len(T)))/(len(X)*len(T))    

def plot_approximations(u_approx, x):
    approximations = ['2d1', '2d2', '3d2']
    for j in range(len(u_approx)):
        fig = go.Figure()
        fig.update_layout(title = approximations[j][0] + " points " +\
                          approximations[j][-1] + \
                          ("-st order" if approximations[j][-1] == '1' else "-nd order")  +\
                          " approximation",
                         xaxis_title = 'x',
                         yaxis_title = 'U(x,t)')
        for i in range(len(u_approx[j])):
            fig.add_trace(go.Scatter(x = x, y = u_approx[j][i], name = 't = ' + str(i)))
        fig.update_layout(template="plotly_dark")
        fig.show()  
        save_plot(fig)  
        
def time_error(x,u,T):
    error = []
    for t in range(len(T)):
        error_tmp = 0
        for i in range(len(x)):
            error_tmp += (u[t][i] - analytic(x[i],T[t]))**2
        error.append(np.sqrt(error_tmp))
    return error

def different_time(method,K,N,Tend,approx_type,len = 50):
    u = []
    time_step_local = []
    space_step = math.pi/(N-1)
    for K_ in range(K,K+len):
        time_step_local.append(Tend/(K_-1))
        u.append(method(space_step, time_step_local[-1], N, K_, approx_type))
    return u, time_step_local

def save_plot(fig):
  global co
  fig.write_html("plots/plot" + str(co)+".html")
  co += 1
  return "plot " + str(co) + " saved"

def different_h(method, K, N, tau, approx_type, len = 30):
    global co
    u = []
    space_steps_local = []
    taus = []
    for N_ in range(N, N+len):
        co += 1
 #       print(co*"= ")
        space_steps_local.append(math.pi/(N_-1))
        taus.append((tau*(K-1))/(100*N_))
        #*чтобы условие куранта выполнялось
        u.append(method(space_steps_local[-1], taus[-1], N_, K = 100*N_, approx_type = approx_type))
    return u, taus

def space_error(x, u, T):
    error = []
    for x_ in range(len(x)):
        error_tmp = 0
        for t in range(len(T)):
            error_tmp += (u[t][x_] - analytic(x[x_], T[t]))**2
        error.append(math.sqrt(error_tmp))
    return error

def different_time_errors(u, K, N, T, x, t, len = 25):
    time_errors = []
    time_steps_local = []
    for u_, K_ in zip(u, range(K, K+len)):
        time_steps_local.append(T/(K_-1))
        t_local = np.arange(0, T + time_steps_local[-1] - 0.01, time_steps_local[-1])
        time_errors.append(time_error(x,u_,t_local))
    return time_errors

def different_space_errors(u, K, N, T, x, t, len = 50):
    space_errors = []
    for u_, N_, tau in zip(u, range(N, N+len), t):
        h_step_local = math.pi/(N_-1)
        x_local = np.arange(0, math.pi + h_step_local - 0.0001, h_step_local)
        t_local = np.arange(0, T + tau - 0.01, tau)
        space_errors.append(space_error(x_local, u_, t))
    return space_errors


def construct_3d_plot(method, X_e, T_e, method_name):
    approx_e = method(approx_type = '2d2')
    z_plot = []
    for j in range(0, len(X_e), 1):
        tmp = []
        for i in range(0, len(X_e[j]), 1):
            tmp.append(analytic(X_e[j][i], T_e[j][i]))
        z_plot.append(tmp)
    line_marker = dict(color='#0066FF', width=2)
    lines = []
    lines.append(go.Scatter3d(x=X_e[0], y=T_e[0], z=approx_e[0], mode='lines', line=line_marker, name = 'Result', legendgroup=1))
    for i, j, k in zip(X_e[1:], T_e[1:], approx_e[1:]):
        lines.append(go.Scatter3d(x=i, y=j, z=k, mode='lines', line=line_marker, showlegend=False))
    line_marker = dict(color='#ff0000', width=2)
    #my_layout = go.Layout({"title": "Views by publisher", scene = go.Scene(),
    #                       "showlegend": False})
    fig_new = go.Figure(data = lines)
    fig_new.update_layout(template="plotly_dark")
    fig_new.update_scenes(xaxis_title=dict(text = 'x'), yaxis_title = dict(text = 't'), zaxis_title = dict(text = "U(x,t)"))
    fig_new.add_trace(go.Scatter3d(x=X_e[0], y=T_e[0], z=z_plot[0], mode='lines', line=line_marker, name = 'Analytic', legendgroup = 2))
    for i, j, k in zip(X_e[1:], T_e[1:], z_plot[1:]):
        fig_new.add_trace(go.Scatter3d(x=i, y=j, z=k, mode='lines', line=line_marker, showlegend=False))
    fig_new.show()
    save_plot(fig_new)

    print("Total error = ", norm_error(X_e, T_e, approx_e))

In [3]:
def plot_error(errors, type):
    global co
    e_tmp = []
    e_x = []
    for e_ in errors:
        e_x.append(len(e_))
        e_tmp.append(sum(e_)/len(e_))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = e_x, y = e_tmp))
    fig.update_layout(title = 'Errors with different ' + str(type),
                     xaxis_title = "splits", 
                     yaxis_title = "Error",
                      template="plotly_dark")
    fig.show()
    save_plot(fig)

<a name = "step3"></a>
# Аналитическое решение

[К оглавлению](#content)

In [4]:
N = 7
T = 7
K = 70

In [5]:
def fill(space_step, N, K):
    U = np.zeros([K,N])
    for j in range(N):
        U[0][j] = f_4(j * space_step)
    return U

In [6]:
h = math.pi/(N-1)
x = [0 + h*i for i in range(N)]
u_a = np.zeros((K,N))
t = 0
tau = T/(K-1)
t = [0 + tau * i for i in range(K)]
approximations = ['2d1', '2d2', '3d2']
if tau/h**2 >= 1/2:
    print("\033[33m {}" .format('Attention!!! Courant condition:'))
    print('sigma > 1/2')
for k in range(K):
    for j in range(N):
        u_a[k][j] = analytic(j * h, k * tau)
fig = go.Figure()
fig.add_trace(go.Scatter(x = x, y = u_a[-1], name = 'Analytical Solution'))
fig.update_layout(title="Analytical Solution",
                  xaxis_title="x",
                  yaxis_title="$U(x,t)$",
                  template="plotly_dark")
fig.show()

for i in range(len(u_a)):
  fig.add_trace(go.Scatter(x = x, y = u_a[i], name = 't = ' + str(round(t[i],2))))

fig.show()

save_plot(fig)

'plot 1 saved'

In [7]:
X_e = [x,x]
T_e = []
T_e.append([0.0 for _ in x])
T_e.append([tau for _ in x])
for k in range(1, K-1):
    X_e.append(x)
    T_e.append([t[k] for _ in x])

<a name = "step4"></a>
# Явная конечно-разностная схема

[К оглавлению](#content)

<a name = "step4.1"></a>
### Теоретическая часть

Два временных слоя: нижний, на котором известно распределение функции (в нашем случае $f_4$ или $\sin x$, и верхний, на котором распределение подлежит определению. Введем сетку и сеточные функции.

Аппроксимируем дифференциальные операторы отношением конечных разностей:
$$\begin{align*}
& u_{t}(x_{j}, t_{k}) = \frac{u_{j}^{k+1}-u_{j}^{k}}{\tau} \\
& u_{xx}(x_j, t_{k}) = \frac{u_{j-1}^{k} -2u_{j}^{k} + u_{j+1}^{k}}{h^2}
\end{align*}$$

Подставим полученные выражения в исходное уравнение:

$$\begin{align*} 
\frac{u_{j}^{k+1}-u_{j}^{k}}{\tau}
 = a\frac{u_{j+1}^{k} -2u_{j}^{k} + u_{j-1}^{k}}{h^2}\end{align*}$$

 Причем $u_{0}^{k} = f_2(t^k)$ или $\varphi_{0}(t^{k}) = \exp(-0.5t)$ и $u_{N}^{k} = f_3(t^k)$ или $\varphi_{l}(t^{k}) = -\exp(-0.5t)$. $u^0_j = \psi(x_j) = f_4$

 Введем обозначение $\sigma = \frac{a\tau}{h^2}$ и выразим из предыдущего уравнения $u_j^{k+1}:$
$$ u_{j}^{k+1} = \sigma u_{j-1}^{k} + (1-2\sigma)u_{j}^{k} + \sigma u_{j+1}^k, \hspace{0.5cm} \sigma = \frac{\tau}{h^2},\hspace{0.2cm} j = \overline{1, N-1}, \hspace{0.2cm} k = 0,1,2,\ldots,$$

Схема обладает существенным изъяном -- она перестает быть условно устойчивой при $\sigma > \frac{1}{2}$ -- условие Куранта. Однако удобством является отсутствие необходимости решения СЛАУ.

<a name = "step4.2"></a>

### Реализация

In [8]:
def approxes(method, h, tau, N, K, a = 1.0):
    approximations = ['2d1', '2d2', '3d2']
    u_approx = []
    for approximate_type in approximations:
        u_approx.append(method(h, tau, N, K, approx_type=approximate_type, a = a))
    return u_approx

In [9]:
def explicit(space_step = math.pi/6, time_step = 7/69, N = 7, K = 70, approx_type = '2d1', a = 1.0):
    sigma = 1 / space_step**2
    if time_step/space_step**2 >= 1/2:
        print("\033[33m {}" .format('Attention!!! Courant condition:'))
        print('sigma > 1/2')
    u = fill(space_step, N, K)
    for k in range(K-1):
        for j in range(1, N-1):
            u[k+1][j] = u[k][j] + \
                f_1(j * space_step, k * time_step) * time_step \
                + time_step*sigma*(u[k][j+1] - 2*u[k][j] + u[k][j-1])
        if approx_type == '2d1':
            #двухточечная, первый порядок
            u[k+1][0] = -space_step*f_2(None, (k+1) * time_step) + u[k+1][1]
            u[k+1][-1] = space_step*f_3(None, (k+1)*time_step) + u[k+1][-2]
        if approx_type == '2d2':
            #двухточечная, второй порядок
            #u[k+1][0] = f_2(None, k*time_step)
            #u[k+1][-1] = f_3(None, k*time_step)
            u[k+1][0] = (u[k+1][1] - space_step * f_2(None, (k+1)*time_step) + (space_step**2 / (2*time_step) * u[k][0])) / (1+space_step**2 / (2*time_step))
            u[k+1][-1] = (u[k+1][-2] + space_step*f_3(None, (k+1)*time_step) + space_step**2 / (2*time_step)*u[k][-1]) / (1+space_step**2/(2*time_step))
        if approx_type == '3d2':
            #трехточечная, второй порядок
            u[k+1][0] = -2*space_step/3*f_2(None, k*time_step) + \
                (4*u[k+1][1] - u[k+1][2]) / 3
            u[k+1][-1] = 2*space_step/3*f_3(None, k*time_step) - \
                (u[k+1][-3]-4*u[k+1][-2])/3
    return u

<a name = "step4.3"></a>
### Исследования

In [10]:
u_approx = approxes(explicit, h, tau, N, K)
plot_approximations(u_approx, x)

In [11]:
er = normm(x,t,u_approx[1])
fig_check_tmp = go.Figure()
fig_check_tmp.add_trace(go.Scatter(x = t, y = er))
fig_check_tmp.update_layout(title = "Error",
                            xaxis_title = "t",
                            yaxis_title = r'$\max |u - analytic|$', 
                            template='plotly_dark')
fig_check_tmp.show()
save_plot(fig_check_tmp)

'plot 5 saved'

In [12]:
[MSE(x, t, u_approx[i]) for i in range(3)]

[3.4429445679460787e-06, 2.2884760222895762e-06, 3.4535816035306396e-06]

In [13]:
for i in range(10, 20, 2):
    fig_check = go.Figure()
    fig_check.add_trace(go.Scatter(x = x, y = u_approx[1][i], name = "Solution by explicit finite-difference schema"))
    fig_check.add_trace(go.Scatter(x = x, y = u_a[i], name = "Analytical solution"))
    fig_check.update_layout(title = "Work in " + str(t[i]) + " seconds from " + str(t[0]),
                           xaxis_title = 'x',
                           yaxis_title = 'U(x,t)',
                            template="plotly_dark")
    fig_check.show()
    save_plot(fig_check)


Построим график погрешности при изменении шага по времени и пространству

In [14]:
for i in tqdm(range(len(approximations))):
    tmp_u, tmp_t = different_time(explicit, K, N, T, approximations[i])
    plot_error(different_time_errors(tmp_u, K, N, T, x, t, len = 70), 'time by ' + approximations[i][-1] + ("-st order" if approximations[i][-1] == '1' else "-nd order") +\
                          " approximation (" + approximations[i][0] + " points)")
#u_dif_time = different_time(explicit, K, N, T, '3d2')
#u, t_ = u_dif_time

  0%|          | 0/3 [00:00<?, ?it/s]

При увеличении количества разбиений по времени (уменьшении шага по времени) аппроксимации ведут себя следующим образом 
* Двухточечная первого порядка -- ошибка уменьшается
* Двухточечная второго порядка -- ошибка уменьшается сильнее, чем в первом случае
* Трехточечная второго порядка -- ошибка увеличивается

То есть при выборе малого количества разбиений выгоднее выбирать трехточечную аппроксимацию второго порядка, при выборе среднего количества разбиений -- двухточечную аппроксимацию первого порядка, при выборе большого количества разбиений -- двухточечную второго порядка

In [15]:
co = 0
for i in tqdm(range(len(approximations))):
    tmp_u, tmp_t = different_h(explicit, K, N, tau, approximations[i], 20)
    plot_error(different_space_errors(tmp_u, K, N,T, x, tmp_t, 20), 'space ' +  approximations[i][-1] + ("-st order" if approximations[i][-1] == '1' else "-nd order") +\
                          " approximation (" + approximations[i][0] + " points)")

  0%|          | 0/3 [00:00<?, ?it/s]

При увеличении разбиений по пространству происходят следующие пируэты:
* Двухточечная первого порядка при малых разбиениях показывает худший результат -- высокую ошибку, которая быстро падает при увеличении количечства разбиений по пространству
* Двухточечная второго порядка показывает стабильную работу
* Трехточечная второго порядка в начале показывает плохой результат, затем быстрее стремится к минимизации.

In [16]:
construct_3d_plot(explicit, X_e, T_e, 'explicit')

Total error =  0.10602540993837405


<a name = "step5"></a>

# Неявная конечно-разностная схема

[К оглавлению](#content)

In [17]:
def tma(a,b,c,d,s):
    P = np.zeros(s)
    Q = np.zeros(s)

    P[0] = -c[0] / b[0]
    Q[0] = d[0] / b[0]

    k = s - 1

    for i in range(1, s):
        P[i] = -c[i] / (b[i] + a[i] * P[i - 1])
        Q[i] = (d[i] - a[i] * Q[i - 1]) / (b[i] + a[i] * P[i - 1])
    P[k] = 0
    Q[k] = (d[k] - a[k] * Q[k - 1]) / (b[k] + a[k] * P[k - 1])

    x = np.zeros(s)
    x[k] = Q[k]

    for i in range(s - 2, -1, -1):
        x[i] = P[i] * x[i + 1] + Q[i]
    return x

<a name = "step5.1"></a>

### Теоретическая часть

Неявная конечно-разностная схема получается путем подстановки аппроксимации дифференциального оператора по пространственной компоненте отношением конечных разностных разностей на верхнем временном слое следующей формы:

$$\frac{\partial^2u}{\partial x^2}\bigg|^{k+1}_j = \frac{u_{j+1}^{k+1} - 2 u_{j}^{k+1} + u_{j-1}^{k+1}}{h^2} + O(h^2)$$

в конкретную задачу. В результате подстановки имеем:

$$\begin{aligned} 
&\frac{u_j^{k+1} - u_j^k}{\tau} = a^2\frac{u_{j+1}^{k+1} - 2u^{k+1}_j + u_{j-1}^{k+1}}{h^2} + O(\tau + h^2), \hspace{0.2cm} j = \overline{1, N-1}, \hspace{0.2cm} k = \overline{0, K-1} \\ 
& u_{0}^{k+1} = \varphi_0(t^{k+1}), \ u_N^{k+1} = \varphi_l(t^{k+1}),\hspace{0.2cm} k = \overline{0, K-1}; \hspace{0.2cm} u^0_j = \psi(x_j), \hspace{0.2cm} j = \overline{0,N}
\end{aligned}$$

Преобразуем первое уравнение к пригодному для решения методом прогонки СЛАУ:

$$a_j \cdot u_{j-1}^{k+1} + b_j\cdot u_j^{k+1} + c_j\cdot u_{j+1}^{k+1} = d_j, \hspace{0.5cm} j = \overline{1,N-1},$$

где $a_j = - \sigma$, $b_j = 1 + 2\sigma$, $c_j = -\sigma$, $d_j = u_j^k$.

Проблемой является зависимой от решения СЛАУ, но зато схема является абсолютно устойчивой.

<a name = "step5.2"></a>

### Реализация

In [18]:
def implicit(space_step = math.pi/6, time_step = 7/69, N = 7, K = 70, approx_type = '2d1', a = 1.0, w = 1):
    sigma = 1 / space_step**2
    u = fill(space_step, N, K)
    for k in range(0,K-1):
        a = np.zeros(N)
        b = np.zeros(N)
        c = np.zeros(N)
        d = np.zeros(N)
        for j in range(1, N-1):
            Rpart = sigma*(u[k][j+1] - 2*u[k][j] + u[k][j-1]) 
            a[j] = sigma*w
            b[j] = - (1/time_step + 2 * sigma * w)
            c[j] = sigma * w
            d[j] = -u[k][j]/time_step - (1-w) * Rpart/2 - f_1(j*space_step, k*time_step) * (1-w) - f_1(j*space_step, (k+1)*time_step) * (w)
        if approx_type == '2d2':
            # двухточечная с первым порядком точности
            b[0] = - 1 / space_step
            c[0] = 1 / space_step
            d[0] = f_2(None,(k+1) * time_step)

            a[-1] = - 1 / space_step
            b[-1] = 1 / space_step
            d[-1] = f_3(None, (k+1) * time_step)
        if approx_type == '2d1':
            # двухточечная со вторым порядком точности
            a[0] = - 1 / (2*space_step)
            c[0] = 1 / (2*space_step)
            d[0] = f_2(None, (k+1) * time_step)
            # здесь из 1,2 уравнений - получаем одно, избавляясь от u1
            c[1] = c[1] - a[1]*c[0]/a[0]
            d[1] = d[1] - a[1]*d[0]/a[0]
            a[1] = 0             
            c[-1] =  1 / (2*space_step)
            a[-1] = - 1 / (2*space_step)
            d[-1] = f_3(None, (k+1) * time_step) 
            a[-2] = a[-2] - c[-2]*a[-1]/c[-1]
            d[-2] = d[-2] - c[-2]*d[-1]/c[-1]
            c[-2] = 0
        elif approx_type == '3d2':
            # трехточечная со вторым порядком точности
            alpha = 1
            beta = 0
            delta = 0
            gamma = 1
            param_a = 1
            param_b = 0
            b[0] = 2*param_a**2/space_step + space_step/time_step - (beta/alpha)*(2*param_a - param_b*space_step)
            c[0] = - 2*param_a**2/space_step
            d[0] = (space_step/time_step)*u[k][0] - f_2(None, (k+1) * time_step) * (2*param_a**2 - param_b*space_step)/alpha

            a[-1] = - 2*param_a**2/space_step
            b[-1] = 2*param_a**2/space_step + space_step/time_step + (delta/gamma)*(2*param_a**2 + param_b*space_step)
            d[-1] = (space_step/time_step)*u[k][-1] + f_3(None, (k+1) * time_step) * (2*param_a**2 + param_b*space_step)/gamma
        Y = np.zeros(25)
        if approx_type != '2d1':
            Y = tma(a, b, c, d, N)
        else:
            a_ = [a[i] for i in range(1,len(a)-1)]
            b_ = [b[i] for i in range(1,len(b)-1)]
            c_ = [c[i] for i in range(1,len(c)-1)]
            d_ = [d[i] for i in range(1,len(d)-1)]
            Y_ = tma(a_, b_, c_, d_, N-2)
            Y0 = (d[0] - c[0]*Y_[1])/a[0]
            Ye = (d[-1] - a[-1]*Y_[-2])/c[-1]
            Y = []
            Y.append(Y0)
            for y in Y_:
                Y.append(y)
            Y.append(Ye)
        u[k + 1] = Y

    return u

<a name = "step5.3"></a>

### Исследования

In [19]:
u_approx = approxes(implicit, h, tau, N, K)
plot_approximations(u_approx, x)

In [20]:
er = normm(x,t,u_approx[2])
fig_check_tmp = go.Figure()
fig_check_tmp.update_layout(title = "Error",
                            xaxis_title = "t",
                            yaxis_title = r'$\max |u - analytic|$', 
                            template='plotly_dark')
fig_check_tmp.add_trace(go.Scatter(x = t, y = er))
save_plot(fig_check_tmp)

'plot 68 saved'

In [21]:
[MSE(x, t, u_approx[i]) for i in range(3)]

[9.559883272411793e-05, 4.741388629997466e-07, 2.9994548591173696e-07]

In [22]:
for i in range(0, len(t), int(len(t)/5)):
    fig_check = go.Figure()
    fig_check.add_trace(go.Scatter(x = x, y = u_approx[2][i], name = "Solution by implicit finite-difference schema"))
    fig_check.add_trace(go.Scatter(x = x, y = u_a[i], name = "Analytical solution"))
    fig_check.update_layout(title = "Work in " + str(t[i]) + " seconds from " + str(t[0]),
                           xaxis_title = 'x',
                           yaxis_title = 'U(x,t)')
    fig_check.update_layout(template="plotly_dark")
    fig_check.show()
    save_plot(fig_check)


In [23]:
for i in tqdm(range(len(approximations))):
    tmp_u, tmp_t = different_time(implicit, K, N, T, approximations[i])
    plot_error(different_time_errors(tmp_u, K, N, T, x, t, len = 40), 'time by ' + approximations[i][-1] + ("-st order" if approximations[i][-1] == '1' else "-nd order") +\
                          " approximation (" + approximations[i][0] + " points)")
#u_dif_time = different_time(explicit, K, N, T, '3d2')
##u, t_ = u_dif_time

  0%|          | 0/3 [00:00<?, ?it/s]

* Трехточечная второго порядка показывает лучшие результаты результаты. При увеличении разбиений по времени ошибка растет.
* Двухточечные первого и второго порядка -- похожие тренды, но результат хуже.

In [24]:
co = 0
for i in tqdm(range(len(approximations))):
    tmp_u, tmp_t = different_h(implicit, K, N, tau, approximations[i], 40)
    plot_error(different_space_errors(tmp_u, K, N,T, x, tmp_t, 40), 'space ' +  approximations[i][-1] + ("-st order" if approximations[i][-1] == '1' else "-nd order") +\
                          " approximation (" + approximations[i][0] + " points)")

  0%|          | 0/3 [00:00<?, ?it/s]

В среднем, лучше трехточечная, на малых разбиениях по пространству -- двухточечная второго порядка.

In [25]:
construct_3d_plot(implicit, X_e, T_e, 'implicit')

Total error =  0.032786179964463286


<a name = "step6"></a>

## Схема Кранка-Николсона

[К оглавлению](#content)

<a name = "step6.1"></a>
### Теоретическая часть



Идея возникла из следующего замечания: на возрастающем участке решение неявной схемы завышается по отношению к точному, а явная наоборот. На убывающем участке ситуация обратная. Рассмотрим неявно-явную схему с весами:

$$\frac{u_j^{k+1}- u_j^k}{\tau} = \theta a^2\frac{u_{j+1}^{k+1} - 2u_j^{k+1} + u_{j-1}^{k+1}}{h^2} + (1-\theta)a^2\frac{u_{j+1}^k - 2u_j^k + u_{j-1}^k}{h^2},$$

где $\theta$ -- вес неявной части конечно-разностной схемы, $1-\theta$ -- вес для явной части, причем $0 \leq \theta \leq 1$. При $\theta = 1$ имеем полностью неявную схему, при $\theta = 0$ -- полностью явную схему. при $\theta = \frac{1}{2}$ -- схему Кранка-Николсона.

### Реализация

In [26]:
def krank_nicolson(space_step = math.pi/6, time_step = 7/69, N = 7, K = 70, approx_type = '2d2', a = 1.0):
    # заполняем решение из начального условия (в нулевой момент времени)
    u = fill(space_step, N, K)

    sigma = 1 / space_step**2
    for k in range(0,K-1):
        a = np.zeros(N)
        b = np.zeros(N)
        c = np.zeros(N)
        d = np.zeros(N)

        for j in range(1, N-1):
            # коэффициент тета = 1/2, поэтому метод абсолютно устойчив
            # аппроксимируем уравнение комбинацией явной и неявной разностной схемы
            Rpart = sigma*(u[k][j+1] - 2*u[k][j] + u[k][j-1])/2

            a[j] = sigma/2
            b[j] = - (1/time_step + sigma)
            c[j] = sigma/2
            d[j] = - u[k][j]/time_step - Rpart - f_1(j*space_step,k*time_step)/2 - f_1(j*space_step,(k+1)*time_step)/2
       
        if approx_type == '2d2':
            # двухточечная с первым порядком точности
            b[0] = - 1 / space_step
            c[0] = 1 / space_step
            d[0] = f_2(None, (k+1) * time_step)

            a[-1] = - 1 / space_step
            b[-1] = 1 / space_step
            d[-1] = f_3(None, (k+1) * time_step)
        if approx_type == '2d1':
            # двухточечная со вторым порядком точности

            a[0] = - 1 / (2*space_step)
            c[0] = 1 / (2*space_step)
            d[0] = f_2(None, (k+1) * time_step)
            # здесь из 1,2 уравнений - получаем одно, избавляясь от u1
            c[1] = c[1] - a[1]*c[0]/a[0]
            d[1] = d[1] - a[1]*d[0]/a[0]
            a[1] = 0             

            c[-1] =  1 / (2*space_step)
            a[-1] = - 1 / (2*space_step)
            d[-1] = f_3(None, (k+1) * time_step) 
            # здесь из m-1,m-2 уравнений - получаем одно, избавляясь от um-1
            a[-2] = a[-2] - c[-2]*a[-1]/c[-1]
            d[-2] = d[-2] - c[-2]*d[-1]/c[-1]
            c[-2] = 0
        elif approx_type == '3d2':
            # трехточечная со вторым порядком точности
            alpha = 1
            beta = 0
            delta = 0
            gamma = 1
            param_a = 1
            param_b = 0
            b[0] = 2*param_a/space_step + space_step/time_step - (beta/alpha)*(2*param_a - param_b*space_step)
            c[0] = - 2*param_a/space_step
            d[0] = (space_step/time_step)*u[k][0] - f_2(None, (k+1) * time_step) * (2*param_a - param_b*space_step)/alpha

            a[-1] = - 2*param_a/space_step
            b[-1] = 2*param_a/space_step + space_step/time_step + (delta/gamma)*(2*param_a + param_b*space_step)
            d[-1] = (space_step/time_step)*u[k][-1] + f_3(None, (k+1) * time_step) * (2*param_a + param_b*space_step)/gamma

        if approx_type != '2d1':
            # используем прогонку, так как система имеет трехдиагональный тип
            Y = tma(a, b, c, d, N)
        else:
            # иначе - получаем трехдиагональную матрицу из исходной, путем выбора элементов не включая граничные
            a_ = [a[i] for i in range(1,len(a)-1)]
            b_ = [b[i] for i in range(1,len(b)-1)]
            c_ = [c[i] for i in range(1,len(c)-1)]
            d_ = [d[i] for i in range(1,len(d)-1)]
            Y_ = tma(a_, b_, c_, d_, N-2)

            # первый элемент считаем из решения системы по формуле
            Y0 = (d[0] - c[0]*Y_[1])/a[0]

            # аналогично последний элемент
            Ye = (d[-1] - a[-1]*Y_[-2])/c[-1]
            Y = []
            
            # формируем итоговое решение
            Y.append(Y0)
            for y in Y_:
                Y.append(y)
            Y.append(Ye)
        u[k + 1] = Y

    return u

In [27]:
u_approx_ = approxes(krank_nicolson, h, tau, N, K)
plot_approximations(u_approx_, x)

In [28]:
er = normm(x,t,u_approx[2])
fig_check_tmp = go.Figure()
fig_check_tmp.add_trace(go.Scatter(x = t, y = er))
fig_check_tmp.update_layout(title = "Error",
                            xaxis_title = "t",
                            yaxis_title = r'$\underset{i}{max} |u - analytic|$', 
                            template='plotly_dark')
save_plot(fig_check_tmp)

'plot 128 saved'

In [29]:
[MSE(x, t, u_approx[i]) for i in range(3)]

[9.559883272411793e-05, 4.741388629997466e-07, 2.9994548591173696e-07]

In [30]:
for approx_type in ['2d2', '3d2']:
    tmp_u, tmp_t = different_time(krank_nicolson, K, N, T, approx_type)
    plot_error(different_time_errors(tmp_u, K, N, T, x, t, len = 40), 'time by ' + approx_type[-1] + ("-st order" if approx_type[-1] == '1' else "-nd order") +\
                          " approximation (" + approx_type[0] + " points)")
#u_dif_time = different_time(explicit, K, N, T, '3d2')
#u, t_ = u_dif_time

In [31]:
co = 0
for approx_type in ['2d1', '2d2', '3d2']:
    tmp_u, tmp_t = different_h(krank_nicolson, K, N, tau, approx_type, 20)
    plot_error(different_space_errors(tmp_u, K, N,T, x, tmp_t, 20), 'space ' +  approx_type[-1] + ("-st order" if approx_type[-1] == '1' else "-nd order") +\
                          " approximation (" + approx_type[0] + " points)")

In [32]:
construct_3d_plot(krank_nicolson, X_e, T_e, 'krank_nicolson')

Total error =  0.07937827341737326
